<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

# 실습 내용

- Grid Search로 KNN 알고리즘 모델을 튜닝합니다.


# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [38]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%config InlineBackend.figure_format='retina'

In [39]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/admission_simple.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [40]:
# 상위 몇 개 행 확인
data.head()

,GRE,TOEFL,RANK,SOP,LOR,GPA,RESEARCH,ADMIT
0,337,118,4,4.5,4.5,9.65,1,1
1,324,107,4,4.0,4.5,8.87,1,1
2,316,104,3,3.0,3.5,8.00,1,0
3,322,110,3,3.5,2.5,8.67,1,1
4,314,103,2,2.0,3.0,8.21,0,0


In [41]:
# 변수 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   GRE       500 non-null    int64  
 1   TOEFL     500 non-null    int64  
 2   RANK      500 non-null    int64  
 3   SOP       500 non-null    float64
 4   LOR       500 non-null    float64
 5   GPA       500 non-null    float64
 6   RESEARCH  500 non-null    int64  
 7   ADMIT     500 non-null    int64  
dtypes: float64(3), int64(5)
memory usage: 31.4 KB


In [42]:
# 기초통계량 확인
data.describe()

,GRE,TOEFL,RANK,SOP,LOR,GPA,RESEARCH,ADMIT
count,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.000000
mean,316.472000,107.192000,3.114000,3.374000,3.48400,8.576440,0.560000,0.436000
std,11.295148,6.081868,1.143512,0.991004,0.92545,0.604813,0.496884,0.496384
min,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.000000
25%,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.000000
50%,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.000000
75%,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,1.000000
max,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,1.000000


# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) 가변수화**

In [43]:
# 가변수화
cols = ['RANK']
data = pd.get_dummies(data, columns=cols, drop_first=True)

**2) x, y 분리**

In [44]:
# target 확인
target = 'ADMIT'

# 데이터 분리
x = data.drop(target, axis = 1)
y = data[target] 

**3) 정규화**

In [45]:
# 정규화
cols = ['GRE', 'TOEFL', 'SOP', 'LOR', 'GPA']
x[cols] = (x[cols] - x[cols].min()) / (x[cols].max() - x[cols].min())

# 확인
x.head()

,GRE,TOEFL,SOP,LOR,GPA,RESEARCH,RANK_2,RANK_3,RANK_4,RANK_5
0,0.94,0.928571,0.875,0.875,0.913462,1,0,0,1,0
1,0.68,0.535714,0.750,0.875,0.663462,1,0,0,1,0
2,0.52,0.428571,0.500,0.625,0.384615,1,0,1,0,0
3,0.64,0.642857,0.625,0.375,0.599359,1,0,1,0,0
4,0.48,0.392857,0.250,0.500,0.451923,0,1,0,0,0


**4) 학습용, 평가용 데이터 분리**

In [46]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 7:3으로 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2022)

# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.

In [47]:
# 1단계: 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [48]:
# 2단계: 선언하기
model = KNeighborsClassifier(n_neighbors=5)

In [49]:
# 3단계: 학습하기
model.fit(x_train, y_train)

KNeighborsClassifier()

In [50]:
# 4단계: 예측하기
y_pred = model.predict(x_test)

In [51]:
# 5단계: 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[75  8]
 [10 57]]
              precision    recall  f1-score   support

           0       0.88      0.90      0.89        83
           1       0.88      0.85      0.86        67

    accuracy                           0.88       150
   macro avg       0.88      0.88      0.88       150
weighted avg       0.88      0.88      0.88       150



**성능에 대한 고민**

- 위 모델링 과정에서 n_neighbors=5를 지정했습니다.
- n_neighbors를 다른 값으로 지정하면 성능이 더 좋을 수도 있습니다.
- 경험이 없는 상태에서 최적의 파라미터를 지정하기는 쉽지 않습니다.

# 5.성능 튜닝

- GridSearchCV를 사용해 최선의 하이퍼파라미터를 갖는 모델을 만듭니다.

## 5.1.모델 만들기

In [52]:
# 1단계: 불러오기
from sklearn.model_selection import GridSearchCV

- 다음 두 가지 모델을 선언합니다.
    - 기본 모델: 기본 알고리즘을 사용하는 튜닝 대상 모델
    - 최종 모델: GridSearchCV 알고리즘을 사용하는 모델
- 다음 정보를 최종 모델에 파라미터로 전달합니다.
    - 기본 모델 이름
    - 파라미터 변수
    - cv: K-Fold 분할 개수(기본값=5)
    - scoring: 평가 방법

In [53]:
# 2단계: 선언하기
model_knn = KNeighborsClassifier()

# 파라미터 선언
  # n_neighbors: range(5, 51)
param = {'n_neighbors': range(5, 51)}

# Grid Search 모델 선언하기
  # cv=10
  # scoring='accuracy'
model = GridSearchCV(model_knn,
                     param,
                     cv=10,
                     scoring='accuracy')

In [54]:
# 3단계: 학습하기
model.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(5, 51)}, scoring='accuracy')

## 5.2.결과 확인

- model.cv_results_ 속성에 성능 테스트와 관련된 많은 정보가 포함되어 있습니다.
- 데이터프레임으로 만들어 확인하면 좋습니다.

In [55]:
# 데이터프레임 만들기
result = pd.DataFrame(model.cv_results_)
result.sort_values(by='mean_test_score', ascending=False, inplace=True)
result.reset_index(drop=True, inplace=True)
result[['param_n_neighbors', 'mean_test_score']].head(10)

,param_n_neighbors,mean_test_score
0,32,0.845714
1,22,0.842857
2,12,0.842857
3,26,0.842857
4,24,0.842857
5,13,0.840000
6,6,0.840000
7,36,0.840000
8,34,0.840000
9,38,0.840000


- model.best_params_, model.best_score_ 값이 최적의 파라미터, 성능입니다.

In [56]:
# 선택된 파라미터와 성능 확인
print(model.best_params_)
print(model.best_score_)

{'n_neighbors': 32}
0.8457142857142855


## 5.3.예측과 평가

- 학습을 통해 예상한 성능과 실제 평가에 따른 성능은 차이가 있을 수 있습니다.
- 예선전에서 성적이 좋았다고 본선에서도 성적이 좋다고 보장할 수 없는 없겠지요?

In [57]:
# 4단계: 예측하기
y_pred = model.predict(x_test)

In [58]:
# 5단계: 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[76  7]
 [11 56]]
              precision    recall  f1-score   support

           0       0.87      0.92      0.89        83
           1       0.89      0.84      0.86        67

    accuracy                           0.88       150
   macro avg       0.88      0.88      0.88       150
weighted avg       0.88      0.88      0.88       150

